# Food Image Classifcation

### Dataset

We are going learn [food images](https://www.kaggle.com/kmader/food41) from kaggle

### Use the kaggle-cli to download the image

```kaggle datasets download -d kmader/food41```

In [36]:
import os

In [39]:
HOME = os.environ["HOME"]
HOME

'/home/paperspace'

In [40]:
DATA  = HOME+"/.kaggle/datasets/kmader/food41/"
DATA

'/home/paperspace/.kaggle/datasets/kmader/food41/'

In [20]:
!!du -sh {DATA}*

['5.3G\t/home/paperspace/.kaggle/datasets/kmader/food41/food41.zip',
 '343M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n1000_r384x384x3.h5',
 '18M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r32x32x1.h5',
 '30M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r32x32x3.h5',
 '68M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r64x64x1.h5',
 '115M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_c101_n10099_r64x64x3.h5',
 '25M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r128x128x1.h5',
 '44M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r128x128x3.h5',
 '1.9M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r32x32x1.h5',
 '3.2M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r32x32x3.h5',
 '6.7M\t/home/paperspace/.kaggle/datasets/kmader/food41/food_test_c101_n1000_r64x64x1.h5',
 '12M\t/home/paperspace/.kaggle/datasets/

In [23]:
%mkdir -p {DATA}images
%mkdir -p {DATA}meta
%ls -l {DATA}

total 11240352
-rw-rw-r-- 1 paperspace paperspace 5681757245 Aug  2 01:45 food41.zip
-rw-rw-r-- 1 paperspace paperspace  359407496 Aug  2 01:45 food_c101_n1000_r384x384x3.h5
-rw-rw-r-- 1 paperspace paperspace   18236331 Aug  2 01:45 food_c101_n10099_r32x32x1.h5
-rw-rw-r-- 1 paperspace paperspace   30452874 Aug  2 01:45 food_c101_n10099_r32x32x3.h5
-rw-rw-r-- 1 paperspace paperspace   70546750 Aug  2 01:45 food_c101_n10099_r64x64x1.h5
-rw-rw-r-- 1 paperspace paperspace  120306214 Aug  2 01:45 food_c101_n10099_r64x64x3.h5
-rw-rw-r-- 1 paperspace paperspace   26042428 Aug  2 01:45 food_test_c101_n1000_r128x128x1.h5
-rw-rw-r-- 1 paperspace paperspace   45897229 Aug  2 01:45 food_test_c101_n1000_r128x128x3.h5
-rw-rw-r-- 1 paperspace paperspace    1938115 Aug  2 01:45 food_test_c101_n1000_r32x32x1.h5
-rw-rw-r-- 1 paperspace paperspace    3294763 Aug  2 01:45 food_test_c101_n1000_r32x32x3.h5
-rw-rw-r-- 1 paperspace paperspace    6968764 Aug  2 01:45 food_test_c101_n1000_r64x64x1.h5
-rw-rw-r--

Unzip and calculate the line of sub-folders: categories

In [24]:
!!cd {DATA}/images; unzip ../images.zip > unzip.log;rm -rf unzip.log; ls -l|wc -l

['102']

In [29]:
!!cd {DATA}/meta; unzip -q ../meta.zip; ls -l meta

['total 4140',
 '-rw-rw-r-- 1 paperspace paperspace    1184 Jul  9  2014 classes.txt',
 '-rw-rw-r-- 1 paperspace paperspace    1184 Sep 23  2013 labels.txt',
 '-rw-rw-r-- 1 paperspace paperspace  566868 Sep 23  2013 test.json',
 '-rw-rw-r-- 1 paperspace paperspace  489429 Sep 23  2013 test.txt',
 '-rw-rw-r-- 1 paperspace paperspace 1697751 Sep 21  2013 train.json',
 '-rw-rw-r-- 1 paperspace paperspace 1468812 Sep 21  2013 train.txt']

In [44]:
META = DATA + "meta/meta/"
META

'/home/paperspace/.kaggle/datasets/kmader/food41/meta/meta/'

In [49]:
for m_file in os.listdir(META):
    print(m_file, "preview","="*20)
    print(open(META+m_file).read()[:50])

train.json preview ====================
{"churros": ["churros/1004234", "churros/1013460",
classes.txt preview ====================
apple_pie
baby_back_ribs
baklava
beef_carpaccio
be
train.txt preview ====================
apple_pie/1005649
apple_pie/1014775
apple_pie/1026
test.json preview ====================
{"churros": ["churros/1061830", "churros/1064042",
test.txt preview ====================
apple_pie/1011328
apple_pie/101251
apple_pie/10343
labels.txt preview ====================
Apple pie
Baby back ribs
Baklava
Beef carpaccio
Be


### Datasets

In [51]:
IMG = DATA+"images/"
IMG

'/home/paperspace/.kaggle/datasets/kmader/food41/images/'

In [52]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms

In [55]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225]),
])

In [56]:
img_set = ImageFolder(IMG,transform = transform, )

#### Train /Valid Split

In [57]:
import numpy as np

In [65]:
trn_set = ImageFolder(IMG,transform = transform, )
val_set = ImageFolder(IMG,transform = transform, )

In [69]:
val_pick = np.random.rand(len(img_set.samples))>0.8
trn_pick = ~val_pick

In [70]:
trn_set.samples = np.array(img_set.samples)[trn_pick].tolist()
val_set.samples = np.array(img_set.samples)[val_pick].tolist()

In [71]:
len(trn_set),len(val_set)

(80504, 20496)

#### Test dataset

In [73]:
from torch.utils.data import DataLoader

In [74]:
gen = iter(DataLoader(trn_set))
next(gen)

[tensor([[[[-1.1932, -1.1589, -1.1418,  ..., -1.4672, -1.4672, -1.3644],
           [-1.5357, -1.5014, -1.4329,  ..., -1.5185, -1.6384, -1.6555],
           [-1.4329, -1.4500, -1.4500,  ..., -1.4500, -1.4843, -1.6042],
           ...,
           [-1.2103, -1.2274, -1.2617,  ..., -1.3473, -1.3302, -1.3473],
           [-1.1932, -1.0562, -0.9534,  ..., -1.3644, -1.3644, -1.3644],
           [-1.0390, -0.9705, -0.9363,  ..., -1.3987, -1.3987, -1.4158]],
 
          [[-1.0728, -1.0553, -1.0378,  ..., -1.3704, -1.3704, -1.2829],
           [-1.2829, -1.2479, -1.2129,  ..., -1.3529, -1.4580, -1.4755],
           [-1.1078, -1.1429, -1.1779,  ..., -1.2654, -1.2829, -1.4055],
           ...,
           [-0.9853, -1.0728, -1.1954,  ..., -1.3179, -1.3179, -1.3354],
           [-0.9678, -0.9153, -0.9503,  ..., -1.3354, -1.3529, -1.3529],
           [-0.8277, -0.8452, -0.9678,  ..., -1.3704, -1.3880, -1.4055]],
 
          [[-1.0027, -0.9678, -0.9504,  ..., -1.2293, -1.2467, -1.1421],
           [-

### Basic CNN

In [75]:
from torch import nn

In [79]:
def conv_layer(in_, out_, ks=3, activation = nn.ReLU()):
    return nn.Sequential(*[
        nn.Conv2d(in_, out_, kernel_size=ks, padding=ks//3, bias = False),
        nn.BatchNorm2d(out_,),
        activation,
    ])

def conv_block(in_,out_,nb_layers):
    layers = []
    for i in range(nb_layers):
        if i == 0: layers.append(conv_layer(in_,out_))
        elif i == nb_layers - 1:layers.append(conv_layer(out_,out_,activation = nn.MaxPool2d((2,2))))
        else: layers.append(conv_layer(out_,out_))
    return nn.Sequential(*layers)

In [ ]:
class cnn(nn.Module):
    def __init__(self):
        self.vgg = conv_block(3,32,2)
        

## Transfer Learning

### VGG

### ResNet